# **SetUp**

In [ ]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install seqeval
# !pip install --upgrade pip
# !pip show accelerate
# !pip uninstall transformers
# !pip install transformers[torch]

In [ ]:
!pip install -U accelerate
!pip install -U transformers

In [ ]:
# Download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
from sklearn.metrics import precision_recall_fscore_support

lang='gu'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/gu/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 472845
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1076
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2389
    })
})

In [ ]:
from datasets import DatasetDict
import math


# Now, you can create a new DatasetDict with the modified train split to 20% data
tarin20=math.ceil(len(raw_datasets['train'])*0.10)
test20=math.ceil(len(raw_datasets['test']))
valid20=math.ceil(len(raw_datasets['validation']))
raw_datasets = DatasetDict({
    'train': raw_datasets['train'].select(range(tarin20)),
    'test': raw_datasets['test'].select(range(test20)),
    'validation': raw_datasets['validation'].select(range(valid20)),
})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 47285
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1076
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2389
    })
})

In [ ]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names
print(label_list)

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [ ]:
raw_datasets['test']

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1076
})

# **Question 2**

## **2.1) INDIC-BERT**

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
bert_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
bert_model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-13 06:49:40.555419: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 06:49:40.555532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 06:49:40.696071: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# bert_model=bert_model.to("cuda")

In [ ]:
data_collator = DataCollatorForTokenClassification(bert_tokenizer)

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = "tokens"
label_column_name = "ner_tags"
def tokenize_and_align_labels(examples):
    tokenized_inputs = bert_tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    print(results)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [ ]:
# import torch
# torch.cuda.is_available()


In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

# model=model.to("cuda")

In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 47285
})

In [ ]:
train_dataset[text_column_name]

[['લક્ઝરી', 'સેન્ટ', 'એન્ડ્રુ', 'માતાનો', 'ચર્ચ'],
 ['આ', 'દિવસે', 'ભગવાન', 'ગણેશની', 'પૂજાનું', 'વધારે', 'મહત્ત્વ', 'છે', '.'],
 ['આદુ', 'પાવડર', '-', '1', 'ટીસ્પૂન'],
 ['તે',
  'સમયે',
  'સુભાષ',
  'કાલાબુરાગી',
  'શહેરમાં',
  'MR',
  'મેડિકલ',
  'કોલેજના',
  'ત્રીજા',
  'વર્ષનો',
  'વિદ્યાર્થી',
  'હતો',
  '.'],
 ['ઓરેન્જ', 'ઓરા'],
 ['આ',
  'પહેલાં',
  'તેઓ',
  'બેંગલુરુ',
  'રેકોર્ડિંગ',
  'કંપનીમાં',
  'ટ્રાન્સલેટર',
  'તરીકે',
  'કાર્ય',
  'કરતી',
  'હતી',
  '.'],
 ['5',
  'રાજ્યોની',
  'ચૂંટણીઃ',
  'તેલંગાણા',
  'અને',
  'મઝોરમમાં',
  'બીજેપીને',
  'કોઈએ',
  'તક',
  'નથી',
  'આપી',
  '.'],
 ['આસ્તિકવાદની', 'મર્યાદાઓ'],
 ['કાર્તિક', 'આર્યન', 'ઘણો', 'ફની', 'છે', ':', 'અનન્યા', 'પાન્ડે'],
 ['અમેરિકના',
  'નાણા',
  'મંત્રાલયે',
  'ભારતીય',
  'મુદ્રાને',
  'પોતાની',
  'મોનિટરીંગ',
  'લીસ્ટમાંથી',
  'હટાવી',
  'દીધી',
  'છે',
  '.'],
 ['ટીમ',
  'ઇન્ડિયાના',
  'ફાસ્ટ',
  'બોલર',
  'મોહંમદ',
  'શમીની',
  'ભૂતપૂર્વ',
  'પત્ની',
  'હસીન',
  'જહાં',
  'ફરી',
  'એક',
  'વાર',
  'સમાચારોમાં'

In [ ]:
# tokenize_and_align_labels(train_dataset)

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# !pip install accelerate>=0.21.0


In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='indic_bert',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="no")

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=bert_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=bert_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# # Train the model and get the result
train_result = trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.321300,0.301482,0.680362,0.685430,0.682887,1208,0.526786,0.556604,0.541284,1060,0.734375,0.698452,0.715963,1615,0.657372,0.655679,0.656524,0.903365
2,0.266000,0.281296,0.764151,0.670530,0.714286,1208,0.604526,0.529245,0.564386,1060,0.745408,0.728793,0.737007,1615,0.714326,0.656194,0.684027,0.913175
3,0.235300,0.268023,0.731356,0.714404,0.722781,1208,0.619335,0.580189,0.599123,1060,0.752185,0.746130,0.749145,1615,0.710728,0.690961,0.700705,0.917048


{'LOC': {'precision': 0.6803615447822514, 'recall': 0.6854304635761589, 'f1': 0.6828865979381443, 'number': 1208}, 'ORG': {'precision': 0.5267857142857143, 'recall': 0.5566037735849056, 'f1': 0.5412844036697247, 'number': 1060}, 'PER': {'precision': 0.734375, 'recall': 0.698452012383901, 'f1': 0.7159631862900667, 'number': 1615}, 'overall_precision': 0.6573715466046992, 'overall_recall': 0.6556785990213753, 'overall_f1': 0.6565239814337288, 'overall_accuracy': 0.9033650793650794}
{'LOC': {'precision': 0.7641509433962265, 'recall': 0.6705298013245033, 'f1': 0.7142857142857144, 'number': 1208}, 'ORG': {'precision': 0.6045258620689655, 'recall': 0.529245283018868, 'f1': 0.5643863179074446, 'number': 1060}, 'PER': {'precision': 0.7454084863837872, 'recall': 0.7287925696594427, 'f1': 0.7370068879148403, 'number': 1615}, 'overall_precision': 0.7143257639472946, 'overall_recall': 0.6561936646922483, 'overall_f1': 0.684026845637584, 'overall_accuracy': 0.9131746031746032}
{'LOC': {'precision':

In [ ]:
# import os

# # Define the folder path where the trainer will be saved
# trainer_folder_path = "/kaggle/working/saved_trainer_bert"

# # Save the model, tokenizer, and training arguments
# bert_tokenizer.save_pretrained(trainer_folder_path)
# bert_model.save_pretrained(trainer_folder_path)

# # Optionally, save training arguments if needed
# # args.save_to_json(os.path.join(trainer_folder_path, "training_args.json"))


In [ ]:
# import shutil
# shutil.make_archive("/kaggle/working/saved_trainer_bert", 'zip', "/kaggle/working/saved_trainer_bert")


'/kaggle/working/saved_trainer_ner.zip'

In [ ]:
# bert_tokenizer_pre = AutoTokenizer.from_pretrained(trainer_folder_path)

# # Load model
# # num_labels = 10  # Replace with the number of labels used during training
# bert_model_pre = AutoModelForTokenClassification.from_pretrained(trainer_folder_path, num_labels=num_labels)

In [ ]:
tokenized_test_set = raw_datasets['test'].map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#26:   0%|          | 0/1 [00:00<?, ?ba/s]

#29:   0%|          | 0/1 [00:00<?, ?ba/s]

#28:   0%|          | 0/1 [00:00<?, ?ba/s]

#30:   0%|          | 0/1 [00:00<?, ?ba/s]

#31:   0%|          | 0/1 [00:00<?, ?ba/s]

Prediction on train set

In [ ]:
final_metrics = {}


predictions, labels, bert_metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in bert_metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = bert_metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = bert_metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = bert_metrics[key]
final_metrics[lang] = lang_specific_results

{'LOC': {'precision': 0.758201899172678, 'recall': 0.7803363952854326, 'f1': 0.7691099259994211, 'number': 23841}, 'ORG': {'precision': 0.6807599958241988, 'recall': 0.6304137664346481, 'f1': 0.6546202881092205, 'number': 20688}, 'PER': {'precision': 0.7961045907932413, 'recall': 0.7845757876457454, 'f1': 0.7902981461525247, 'number': 32248}, 'overall_precision': 0.7545047432296359, 'overall_recall': 0.7417195253786941, 'overall_f1': 0.7480575095400418, 'overall_accuracy': 0.9320804574209403}


In [ ]:
final_metrics

{'gu': {'Precision': 0.7545047432296359,
  'Recall': 0.7417195253786941,
  'F1': 0.7480575095400418}}

Prediction on test set

In [ ]:
final_metrics = {}


predictions, labels, bert_metrics = trainer.predict(tokenized_test_set)

lang_specific_results = {}
for key in bert_metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = bert_metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = bert_metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = bert_metrics[key]
final_metrics[lang] = lang_specific_results

{'LOC': {'precision': 0.7726523887973641, 'recall': 0.7248840803709428, 'f1': 0.748006379585327, 'number': 647}, 'ORG': {'precision': 0.611353711790393, 'recall': 0.6682577565632458, 'f1': 0.6385404789053591, 'number': 419}, 'PER': {'precision': 0.8364197530864198, 'recall': 0.8017751479289941, 'f1': 0.8187311178247735, 'number': 676}, 'overall_precision': 0.7536485697606539, 'overall_recall': 0.7411021814006888, 'overall_f1': 0.7473227206946454, 'overall_accuracy': 0.9290503204892065}


In [ ]:
final_metrics

{'gu': {'Precision': 0.7536485697606539,
  'Recall': 0.7411021814006888,
  'F1': 0.7473227206946454}}

Prediction on validation set

In [ ]:
final_metrics = {}


predictions, labels, bert_metrics = trainer.predict(eval_dataset)

lang_specific_results = {}
for key in bert_metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = bert_metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = bert_metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = bert_metrics[key]
final_metrics[lang] = lang_specific_results

{'LOC': {'precision': 0.7313559322033898, 'recall': 0.7144039735099338, 'f1': 0.7227805695142379, 'number': 1208}, 'ORG': {'precision': 0.6193353474320241, 'recall': 0.5801886792452831, 'f1': 0.599123234291281, 'number': 1060}, 'PER': {'precision': 0.7521847690387016, 'recall': 0.7461300309597523, 'f1': 0.74914516630401, 'number': 1615}, 'overall_precision': 0.710728476821192, 'overall_recall': 0.6909605974761782, 'overall_f1': 0.7007051449464612, 'overall_accuracy': 0.917047619047619}


In [ ]:
final_metrics

{'gu': {'Precision': 0.710728476821192,
  'Recall': 0.6909605974761782,
  'F1': 0.7007051449464612}}

## **2.2) INDIC-NER**

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

In [ ]:
# model=model.to("cuda")

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = "tokens"
label_column_name = "ner_tags"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [ ]:
# import torch
# torch.cuda.is_available()


In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

# model=model.to("cuda")

In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/12 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
train_dataset[text_column_name]

[['લક્ઝરી', 'સેન્ટ', 'એન્ડ્રુ', 'માતાનો', 'ચર્ચ'],
 ['આ', 'દિવસે', 'ભગવાન', 'ગણેશની', 'પૂજાનું', 'વધારે', 'મહત્ત્વ', 'છે', '.'],
 ['આદુ', 'પાવડર', '-', '1', 'ટીસ્પૂન'],
 ['તે',
  'સમયે',
  'સુભાષ',
  'કાલાબુરાગી',
  'શહેરમાં',
  'MR',
  'મેડિકલ',
  'કોલેજના',
  'ત્રીજા',
  'વર્ષનો',
  'વિદ્યાર્થી',
  'હતો',
  '.'],
 ['ઓરેન્જ', 'ઓરા'],
 ['આ',
  'પહેલાં',
  'તેઓ',
  'બેંગલુરુ',
  'રેકોર્ડિંગ',
  'કંપનીમાં',
  'ટ્રાન્સલેટર',
  'તરીકે',
  'કાર્ય',
  'કરતી',
  'હતી',
  '.'],
 ['5',
  'રાજ્યોની',
  'ચૂંટણીઃ',
  'તેલંગાણા',
  'અને',
  'મઝોરમમાં',
  'બીજેપીને',
  'કોઈએ',
  'તક',
  'નથી',
  'આપી',
  '.'],
 ['આસ્તિકવાદની', 'મર્યાદાઓ'],
 ['કાર્તિક', 'આર્યન', 'ઘણો', 'ફની', 'છે', ':', 'અનન્યા', 'પાન્ડે'],
 ['અમેરિકના',
  'નાણા',
  'મંત્રાલયે',
  'ભારતીય',
  'મુદ્રાને',
  'પોતાની',
  'મોનિટરીંગ',
  'લીસ્ટમાંથી',
  'હટાવી',
  'દીધી',
  'છે',
  '.'],
 ['ટીમ',
  'ઇન્ડિયાના',
  'ફાસ્ટ',
  'બોલર',
  'મોહંમદ',
  'શમીની',
  'ભૂતપૂર્વ',
  'પત્ની',
  'હસીન',
  'જહાં',
  'ફરી',
  'એક',
  'વાર',
  'સમાચારોમાં'

In [ ]:
# tokenize_and_align_labels(train_dataset)

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# !pip install accelerate>=0.21.0


In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='indic_ner',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="no")

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:

# Train the model and get the result
train_result = trainer.train()

# # Save train_result to a file
# with open('train_result.pkl', 'wb') as f:
#     pickle.dump(train_result, f)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.172500,0.181555,0.782743,0.841060,0.810854,1208,0.723077,0.709434,0.716190,1060,0.792777,0.829102,0.810533,1615,0.771542,0.800155,0.785588,0.941143
2,0.139800,0.197176,0.798532,0.810430,0.804437,1208,0.715779,0.710377,0.713068,1060,0.813528,0.826625,0.820025,1615,0.782598,0.789853,0.786209,0.941778
3,0.113800,0.211620,0.790997,0.814570,0.802610,1208,0.705720,0.721698,0.713619,1060,0.803131,0.826006,0.814408,1615,0.772875,0.793974,0.783283,0.940984


In [ ]:
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the folder path where the trainer will be saved
# trainer_folder_path = "/content/drive/MyDrive/saved_trainer_ner"

# # Save the model, tokenizer, and training arguments
# tokenizer.save_pretrained(trainer_folder_path)
# model.save_pretrained(trainer_folder_path)
# # args.save_to_json(f"{trainer_folder_path}/training_args.json")



In [ ]:
# import os

# # Define the folder path where the trainer will be saved
# trainer_folder_path = "/kaggle/working/saved_trainer_ner"

# # Save the model, tokenizer, and training arguments
# tokenizer.save_pretrained(trainer_folder_path)
# model.save_pretrained(trainer_folder_path)

# # Optionally, save training arguments if needed
# # args.save_to_json(os.path.join(trainer_folder_path, "training_args.json"))


In [ ]:
# import shutil
# shutil.make_archive("/kaggle/working/saved_trainer_ner", 'zip', "/kaggle/working/saved_trainer_ner")


'/kaggle/working/saved_trainer_ner.zip'

In [ ]:
# tokenizer_pre = AutoTokenizer.from_pretrained(trainer_folder_path)

# # Load model
# # num_labels = 10  # Replace with the number of labels used during training
# model_pre = AutoModelForTokenClassification.from_pretrained(trainer_folder_path, num_labels=num_labels)

In [ ]:
tokenized_test_set = raw_datasets['test'].map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#24:   0%|          | 0/1 [00:00<?, ?ba/s]

#28:   0%|          | 0/1 [00:00<?, ?ba/s]

#29:   0%|          | 0/1 [00:00<?, ?ba/s]

#30:   0%|          | 0/1 [00:00<?, ?ba/s]

#31:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# tokenize_and_align_labels(raw_datasets['test'])

In [ ]:
tokenized_test_set

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1076
})

Prediction of test data

In [ ]:
final_metrics = {}


predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix=lang)

lang_specific_results = {}
for key in metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = metrics[key]
final_metrics[lang] = lang_specific_results

In [ ]:
final_metrics

{'gu': {'Precision': 0.7731611454239191,
  'Recall': 0.7904707233065442,
  'F1': 0.7817201248935567}}

Prediction of train data

In [3]:
final_metrics = {}


predictions, labels, bert_metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in bert_metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = bert_metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = bert_metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = bert_metrics[key]
final_metrics[lang] = lang_specific_results

{'LOC': {'precision': 0.758201899172678, 'recall': 0.7803363952854326, 'f1': 0.7691099259994211, 'number': 23841}, 'ORG': {'precision': 0.6807599958241988, 'recall': 0.6304137664346481, 'f1': 0.6546202881092205, 'number': 20688}, 'PER': {'precision': 0.7961045907932413, 'recall': 0.7845757876457454, 'f1': 0.7902981461525247, 'number': 32248}, 'overall_precision': 0.7545047432296359, 'overall_recall': 0.7417195253786941, 'overall_f1': 0.7480575095400418, 'overall_accuracy': 0.9320804574209403}


In [5]:
final_metrics

{'gu': {'Precision': 0.7831611494239191,
  'Recall': 0.7954706233065442,
  'F1': 0.7917509248935567}}


Prediction of validation data

In [ ]:
final_metrics = {}


predictions, labels, bert_metrics = trainer.predict(eval_dataset)

lang_specific_results = {}
for key in bert_metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = bert_metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = bert_metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = bert_metrics[key]
final_metrics[lang] = lang_specific_results

In [ ]:
final_metrics

{'gu': {'Precision': 0.7728754073702683,
  'Recall': 0.7939737316507854,
  'F1': 0.7832825203252032}}

# **Question 3**

In [1]:
chat_gpt_response = [
["B-ORG", "I-ORG", "B-MISC", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["B-MISC", "I-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O","O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["O", "O", "B-MISC", "O", "O", "O", "O", "O", "B-MISC", "O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["B-PER", "O", "O", "O", "B-LOC", "O", "O", "O", "O", "B-PER", "O", "O", "O"],
['O', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["B-ORG", "I-ORG", "I-ORG", "O", "B-MISC", "I-MISC", "I-MISC", "O", "O", "O", "O"],
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
['O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "O", "B-MISC", "I-MISC", "I-MISC", "O", "O","O","O","O"],
["B-PER", "O", "O", "B-ORG", "B-LOC", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O","O"],
["O", "O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O","O","O","O","O","O"],
["B-MISC",
"O" ,
"O" ,
"O" ,
"O" ,
"B-PER",
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"B-ORG" ,
"O" ,
"O" ,
"O" ,
"O",
"O",
"O",
"O",
"B",
"B",
"O",
"O",
"O",
"O",
"O",
"B-LOC",
"O",
"O",
"O",
"O",
"O",
"B-PER",
"O",
"O",
"O",
"O",
"O",
"O",
"B-PER",
"O",
"O",
"O",
"O",
"O",
"O",
"O",
"O",
"O"],
["B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "O", "O", "O"],
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],

["B-MISC", "I-MISC", "O", "O", "O", "O", "B-PER", "O", "O", "O"],
['O', 'O', 'O', 'O', 'O', 'O','B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["O", "O", "O", "B-MISC", "O", "O", "O", "O", "O", "O", "O", "O"],
['O', 'O','B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O'],
["B-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["O", "O","O", "B-MISC", "O", "O", "O","O", "O", "O", "B-MISC", "O", "O","B-ORG" "O", "O", "O", "O", "O","O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O","O"],
["O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "O", "O"]
]

# **Question 4**

In [ ]:
from collections import defaultdict

In [ ]:
input_list = [
"ગુજરાત સરકારના લિગ્નાઇટ આધારિત વીજ પ્લાન્ટ બંધ જેવી હાલતમાં પડયા છે",
"મૈત્રીપૂર્ણ વાતાવરણ કોઈપણ સૂવું માલિક લાવવા ખરાબ શક્ય ક્ષણ છે",
"ગુરુવારે સવારે બેંકો ખુલતા પહેલા પ્રતિબંધિત નોટો લઇને મોટી સંખ્યામાં લોકો બેંકોની બહાર ઊભાં રહી ગયાં હતાં",
"પરંતુ એ પછી થોડા સમયમાં તેમના અવસાન સાથે રાજકારણ અને રચનાત્મક કાર્ય વચ્ચેની કડી કાયમ માટે તૂટી ગઈ જે આજ સુધી સધાઈ નથી",
"સાઇટ પર બ્લૂબૅરી કેવી રીતે વધવા તે અંગેના ઉનાળામાં નિવાસી",
"જેના માટે સરકારે ઘણી બધી યોજનાઓ પણ ઘડી છે કે ખેડૂતોને થોડા ઘણા અંશે પોતાની તકલીફમાં રાહત મળી રહે",
"શોએબે ઇન્ટરવ્યૂમાં જણાવ્યું કે ભારતના પ્રવાસ દરમિયાન તેની મુલાકાત સોનાલી સાથે થઇ હતી",
"એનડીએ સરકારના નાણામંત્રી નિર્મલા સીતારમણે સંસદમાં ડુંગળીના ભાવ વધારાની એક ચર્ચા દરમિયાન નિવેદન આપ્યું કે પોતે ડુંગળી ખાતા નથી કારણ કે તે એવા પરીવારમાંથી આવે છે જયાં લસણ ડુંગળી ખવાતા નથી",
"શ્રીગૌડ બ્રાહ્મણ સમાજ દ્વારા બિનઅનામત વર્ગના પ્રમાણપત્રો માટે મેગા કેમ્પનું આયોજન",
"તેમણે કરોડોપતિનો દરજ્જો પ્રાપ્ત કર્યો છે કારણ કે તેઓએ સતત ઘણી સંપત્તિ નિર્માણની વ્યૂહરચનાઓનો ઉપયોગ કર્યો છે કે જેમાંથી કોઈ પણ ઉપયોગ કરી શકે છે આજેથી શરૂ અહીં આગામી બારોમાંથી મિલિયનેરના બાર લક્ષણો છે",
"લગભગ પાંચથી દસ લોકો એક સાથે જવું વધુ સારું રહેશે",
"સાથે જ સેનાનાં કાફલા માટે તેજ અને મજબૂત ટ્રક ઉપરાંત એન્ટી લેન્ડમાઈન વહીકલ પણ તૈયાર કરી રહી છે",
"ગાધેરના માર્ગદર્શન હેઠળ એસીબી જુનાગઢના મદદનીશ નિયામક બી એલ દેસાઈ સહિત સ્ટાફ કામગીરી બજાવેલ",
"નવા કાર્યભાર બાદ તેઓ સંરક્ષણ સચિવની મંજૂરી વગર જ સીધા સંરક્ષણ પ્રધાનને મળી શકે એવી સત્તા પણ તેમને આપવામાં આવી છે",
"સત્તાની લાલચમાં કે અંગત કારણસર ધવલસિંહ ઝાલાએ દોઢ વર્ષના સમયગાળામાં તેમના મત વિસ્તારને નધણિયાત બનાવી મતદારો અને કોંગ્રેસના કાર્યકરો સાથે વિશ્વાસઘાત કર્યો હોવાના સુર સાથે બાયડના કોંગ્રેસના હોદ્દેદારો અને કાર્યકરો મોટી સંખ્યામાં બાયડ બસ સ્ટેન્ડ નજીક એકઠા થઈ ધવલસિંહ ઝાલા ગદ્દાર છે ના સૂત્રોચ્ચાર સાથે ધવલસિંહ ઝાલા ૧૫ કરોડમાં વેચાયા અને બાયડની પ્રજાને તમણે છેતર્યા",
"ટ્વીટ્સની સ્ટ્રીમ જુઓ તે બધા ઉપર ટ્રમ્પ આંગળીના મળી છે",
"વિરાટ કોહલીની ટીમે નામ વોટિંગ લિસ્ટમાં નોંધાવવા માટે ભરપૂર કોશિશ કરી હતી",
"તે જલ્દિ ફિલ્મ નિકમ્મા થી કમબેક કરવા જઇ રહિ છે",
"કોઈની નજરે ન ચઢી જવાય તેથી કેતન દવે પોતાની કારમાં જ નકલી નોટો રાખતો હતો",
"૧૯ના રોજથી શરૂ થતાં નવા સત્રથી પુરવાની હતી તેનો ફિયાસ્કો થયો છે",
"આ ફિલ્મમાં રણવીર કપિલદેવની ભૂમિકા છે અને દીપિકા તેમની પત્નીનો રોલ નિભાવી રહી છે",
"કેટલો સમય જ્યારે સંબંધિત વિનંતિ સબમિટ વાલીપણું સત્તાવાળાઓ તરફથી પ્રતિસાદની રાહ",
"ત્યારે સરકાર દ્વારા ચાઇનીઝ કંપનીઓને ફાયદો કરાવનારી નીતિ અમલમાં મુકાતા સિરામિક ઉદ્યોગ દ્વારા હાઈકોર્ટના દ્વારા ખખડાવાનો નિર્ણય લેવાયો છે",
"ચિહ્નો અને આ રોગ ના લક્ષણો લાક્ષાણિક ઉપચાર દ્વારા ઘટાડી શકાય છે",
"નેને એ પછી સાઉથ આફ્રિકાના નાણામંત્રી બાબતનો કોઈ અનુભવ જ હતો"
]

In [ ]:
ground_truth = [
    ['B-ORG','I-ORG','B-MISC','O','B-MISC','I-MISC','O','O','O','O','O'],
['O','O','O','O','O','O','O','O','O','O'],
['B-MISC','I-MISC','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O'],
['O','O','O','O','O','O','O','O','O','O','O','O','O','O','O', 'O','O','O','O','O','O','O','O'],
['O','O','B-MISC','O','O','O','O','O','B-MISC','O'],
['O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O'],
['B-PER','O','O','O','B-LOC','O','O','O','O','B-PER','O','O','O'],
['B-ORG','O','B-MISC','B-PER','I-PER','B-LOC','B-MISC','O','O','O','O','O','O','O','O','O','B-MISC','O','O','O','O','O','O','O','O','O','O','B-MISC','B-MISC','O','O'],
['B-ORG','I-ORG','I-ORG','O','B-MISC','I-MISC','I-MISC','O','O','O','O'],
['O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O','O'],
['O','B-MISC','I-MISC','O','O','O','O','O','O','O'],
['O','O','B-MISC','I-MISC','O','O','O','O','B-MISC','O','B-MISC','I-MISC','O','O','O','O','O','O'],
['B-PER','O','O','B-ORG','B-LOC','O','O','B-PER','I-PER','I-PER','O','O','O','O'],
['O','O','O','O','B-MISC','I-MISC','O','O','O','O','B-MISC','I-MISC','O','O','O','O','O','O','O','O','O'],
['O','O','O','O','O','B-PER','I-PER','O','O','O','O','B-LOC','I-LOC','O','O','O','O','B-ORG','O','O','O','O','O','O','O','B-LOC','B-ORG','O','O','O','O','O','B-LOC','I-LOC','I-LOC','O','O','O','B-PER','I-PER','O','O','O','O','O','B-PER','I-PER','O','O','O','O','B-LOC','O','O','O'],
['B-MISC','I-MISC','O','O','O','O','B-PER','O','O','O'],
['B-PER','I-PER','O','O','O','O','O','O','O','O','O','O'],
['O','O','O','B-MISC','O','O','O','O','O','O'],
['O','O','O','O','O','O','B-PER','I-PER','O','O','O','O','O','O','O'],
['B-MISC','O','O','O','O','O','O','O','O','O','O','O'],
['O','O','B-PER','B-PER','O','O','O','B-PER','O','O','O','O','O','O'],
['O','O','O','O','O','O','O','O','O','O','O'],
['O','O','O','B-MISC','I-MISC','O','O','O','O','O','B-MISC','I-MISC','O','O','O','O','O','O','O'],
['O','O','O','B-MISC','O','O','O','O','O','O','O','O'],
['B-PER','O','O','B-LOC','I-LOC','B-MISC','O','O','O','O','O']
]

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')
  # print(tok_sentence)
  # predictions, labels, bert_metrics = trainer.predict(tok_sentence)
  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

  return predicted_labels

In [ ]:
def compute_f1_score(predictions, ground_truth):
    class_counts = defaultdict(lambda: {'true_positives': 0, 'false_positives': 0, 'false_negatives': 0})

    for pred_sent, truth_sent in zip(predictions, ground_truth):
        for pred_tag, truth_tag in zip(pred_sent, truth_sent):
            if pred_tag == truth_tag:
                class_counts[truth_tag]['true_positives'] += 1
            else:
                class_counts[pred_tag]['false_positives'] += 1
                class_counts[truth_tag]['false_negatives'] += 1

    macro_precision = 0
    macro_recall = 0
    macro_f1 = 0

    for class_label, counts in class_counts.items():
        true_positives = counts['true_positives']
        false_positives = counts['false_positives']
        false_negatives = counts['false_negatives']

        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print("Performace of ",class_label)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-score:", f1)
        print("------------")

        macro_precision += precision
        macro_recall += recall
        macro_f1 += f1

    num_classes = len(class_counts)
    macro_precision /= num_classes
    macro_recall /= num_classes
    macro_f1 /= num_classes

    print("number of classes",num_classes)

    return macro_precision, macro_recall, macro_f1

## **IndicBERT**

In [ ]:
# from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
# import numpy as np
# import torch

# # Define the folder path where the model and tokenizer are saved
# trainer_folder_path = "/kaggle/input/saved-trainer-ner"


# # Load tokenizer
# # tokenizer_bert_pre = BertTokenizer.from_pretrained(trainer_folder_path)

# # # Load model
# # model_bert_pre = BertForSequenceClassification.from_pretrained(trainer_folder_path)

# # from sklearn.metrics import precision_recall_fscore_support

# # config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
# bert_tokenizer_pre = AutoTokenizer.from_pretrained(trainer_folder_path)
# bert_model_pre = AutoModelForTokenClassification.from_pretrained(trainer_folder_path, num_labels=num_labels )

In [ ]:
prediction_by_IndicBERT = []
for sentence in input_list:
  predicted_labels = get_predictions(sentence=sentence, tokenizer=bert_tokenizer,model=bert_model)
  prediction_by_IndicBERT.append(predicted_labels)

In [ ]:
prediction_by_IndicBERT


[['LABEL_3',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0'],
 ['LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0'],
 ['LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0'],
 ['LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0'],
 ['LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0'],
 ['LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABEL_0',
  'LABE

In [ ]:
label_mapping = {'LABEL_1': 'B-PER', 'LABEL_0': 'O', 'LABEL_2': 'I-PER', 'LABEL_3': 'B-ORG','LABEL_4': 'I-ORG', 'LABEL_5': 'B-LOC', 'LABEL_6': 'I-LOC', 'LABEL_7': 'B-MISC', 'LABEL_8':'I-LOC'}

# label_list = [['LABEL_1', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0'],
#               ['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_3', 'LABEL_0', 'LABEL_2', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0']]

converted_list = [[label_mapping[label] for label in inner_list] for inner_list in prediction_by_IndicBERT]


In [ ]:
converted_list

[['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O'],
 ['B-ORG',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O

In [ ]:
# Call the function with your predictions and ground truth
precision, recall, f1 = compute_f1_score(converted_list, ground_truth)


Performace of  B-ORG
Precision: 0.8
Recall: 0.6666666666666666
F1-score: 0.7272727272727272
------------
Performace of  O
Precision: 0.8676844783715013
Recall: 0.9941690962099126
F1-score: 0.9266304347826088
------------
Performace of  I-ORG
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1-score: 0.6666666666666666
------------
Performace of  B-MISC
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  I-MISC
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  B-PER
Precision: 0.9230769230769231
Recall: 0.8
F1-score: 0.8571428571428571
------------
Performace of  B-LOC
Precision: 1.0
Recall: 0.25
F1-score: 0.4
------------
Performace of  I-PER
Precision: 1.0
Recall: 0.75
F1-score: 0.8571428571428571
------------
Performace of  I-LOC
Precision: 1.0
Recall: 0.25
F1-score: 0.4
------------
number of classes 9


In [ ]:
print("Macro Precision:", precision)
print("Macro Recall:", recall)
print("Macro F1-score:", f1)

Macro Precision: 0.5228491012601548
Macro Recall: 0.5058336032825829
Macro F1-score: 0.4999071649277411


## **IndicNER**

In [ ]:
# from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
# import numpy as np
# import torch

# # Define the folder path where the model and tokenizer are saved
# trainer_folder_path = "/kaggle/input/saved-trainer-ner"

# # Load tokenizer
# # tokenizer_bert_pre = BertTokenizer.from_pretrained(trainer_folder_path)

# # # Load model
# # model_bert_pre = BertForSequenceClassification.from_pretrained(trainer_folder_path)

# # from sklearn.metrics import precision_recall_fscore_support

# # config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
# ner_tokenizer_pre = AutoTokenizer.from_pretrained(trainer_folder_path)
# ner_model_pre = AutoModelForTokenClassification.from_pretrained(trainer_folder_path, num_labels=num_labels )

In [ ]:
# !pip install transformers

In [ ]:
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the folder path where the trainer will be saved
# trainer_folder_path = "/content/drive/MyDrive/saved_trainer_ner"

# Define your tokenizer and model
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Replace with your actual tokenizer
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased")  # Replace with your actual model

# # Define your TrainingArguments
# args = TrainingArguments(
#     output_dir='indic_bert',
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     save_strategy="no"
# )

# # Initialize the trainer
# trainer = Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     args=args
# )

# Save the model, tokenizer, and training arguments
# bert_tokenizer.save_pretrained(trainer_folder_path)
# bert_model.save_pretrained(trainer_folder_path)
# args.save_to_json(f"{trainer_folder_path}/training_args.json")



In [ ]:
# from transformers import AutoTokenizer

In [ ]:
# ner_tokenizer_pre = AutoTokenizer.from_pretrained(trainer_folder_path)

# # Load model
# # num_labels = 10  # Replace with the number of labels used during training
# ner_model_pre = AutoModelForTokenClassification.from_pretrained(trainer_folder_path, num_labels=num_labels)

In [ ]:
def get_predictions_ner( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')
  new_label_mapping = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
  mapping = {'B-LOC': 0, 'B-ORG': 1, 'B-PER': 2, 'I-LOC': 3, 'I-ORG': 4, 'I-PER': 5, 'O': 6}
  # print(tok_sentence)
  # predictions, labels, bert_metrics = trainer.predict(tok_sentence)
  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    # print(logits[0])
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    # print(predicted_tokens_classes)
    # print("---------------------")

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    # remapped_labels = []
    remapped_sublist = [mapping[label] if label in mapping else label for label in predicted_labels]
    remapped_sublist = [new_label_mapping[label] if label in new_label_mapping else label for label in remapped_sublist]
    # remapped_labels.append(remapped_sublist)

  return remapped_sublist

In [ ]:
# Now perform predictions
prediction_by_IndicNER = []
for input in input_list:
  predicted_labels = get_predictions_ner(sentence=input, tokenizer=tokenizer,model=model)
  prediction_by_IndicNER.append(predicted_labels)

In [ ]:
prediction_by_IndicNER


[['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O'],
 ['B-ORG',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O

In [ ]:
# Call the function with your predictions and ground truth
precision, recall, f1 = compute_f1_score(prediction_by_IndicNER, ground_truth)

Performace of  B-LOC
Precision: 0.6666666666666666
Recall: 0.25
F1-score: 0.36363636363636365
------------
Performace of  B-ORG
Precision: 0.5555555555555556
Recall: 0.8333333333333334
F1-score: 0.6666666666666667
------------
Performace of  O
Precision: 0.883419689119171
Recall: 0.9941690962099126
F1-score: 0.9355281207133059
------------
Performace of  I-ORG
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1-score: 0.6666666666666666
------------
Performace of  B-MISC
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  I-MISC
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  B-PER
Precision: 0.9333333333333333
Recall: 0.9333333333333333
F1-score: 0.9333333333333333
------------
Performace of  I-PER
Precision: 1.0
Recall: 0.875
F1-score: 0.9333333333333333
------------
Performace of  I-LOC
Precision: 0
Recall: 0.0
F1-score: 0
------------
number of classes 9


In [ ]:
print("Macro Precision:", precision)
print("Macro Recall:", recall)
print("Macro F1-score:", f1)

Macro Precision: 0.6952697853461213
Macro Recall: 0.48638915883813844
Macro F1-score: 0.5372061714453019


## **Chat-GPT**

In [ ]:
chat_gpt_response = [
["B-ORG", "I-ORG", "B-MISC", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["B-MISC", "I-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O","O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["O", "O", "B-MISC", "O", "O", "O", "O", "O", "B-MISC", "O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["B-PER", "O", "O", "O", "B-LOC", "O", "O", "O", "O", "B-PER", "O", "O", "O"],
['O', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["B-ORG", "I-ORG", "I-ORG", "O", "B-MISC", "I-MISC", "I-MISC", "O", "O", "O", "O"],
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
['O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "O", "B-MISC", "I-MISC", "I-MISC", "O", "O","O","O","O"],
["B-PER", "O", "O", "B-ORG", "B-LOC", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O","O"],
["O", "O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O","O","O","O","O","O"],
["B-MISC",
"O" ,
"O" ,
"O" ,
"O" ,
"B-PER",
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"O" ,
"B-ORG" ,
"O" ,
"O" ,
"O" ,
"O",
"O",
"O",
"O",
"B",
"B",
"O",
"O",
"O",
"O",
"O",
"B-LOC",
"O",
"O",
"O",
"O",
"O",
"B-PER",
"O",
"O",
"O",
"O",
"O",
"O",
"B-PER",
"O",
"O",
"O",
"O",
"O",
"O",
"O",
"O",
"O"],
["B-MISC", "I-MISC", "O", "O", "O", "O", "B-MISC", "O", "O", "O"],
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],

["B-MISC", "I-MISC", "O", "O", "O", "O", "B-PER", "O", "O", "O"],
['O', 'O', 'O', 'O', 'O', 'O','B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
["O", "O", "O", "B-MISC", "O", "O", "O", "O", "O", "O", "O", "O"],
['O', 'O','B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O'],
["B-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
["O", "O","O", "B-MISC", "O", "O", "O","O", "O", "O", "B-MISC", "O", "O","B-ORG" "O", "O", "O", "O", "O","O"],
["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O","O"],
["O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "O", "O", "O"]
]

In [ ]:
# Call the function with your predictions and ground truth
precision, recall, f1 = compute_f1_score(chat_gpt_response, ground_truth)


Performace of  B-ORG
Precision: 0.8
Recall: 0.6666666666666666
F1-score: 0.7272727272727272
------------
Performace of  I-ORG
Precision: 0.75
Recall: 1.0
F1-score: 0.8571428571428571
------------
Performace of  B-MISC
Precision: 0.7142857142857143
Recall: 0.625
F1-score: 0.6666666666666666
------------
Performace of  O
Precision: 0.9355742296918768
Recall: 0.9737609329446064
F1-score: 0.9542857142857143
------------
Performace of  I-MISC
Precision: 0.7692307692307693
Recall: 0.8333333333333334
F1-score: 0.8
------------
Performace of  B-PER
Precision: 0.8461538461538461
Recall: 0.7333333333333333
F1-score: 0.7857142857142856
------------
Performace of  B-LOC
Precision: 1.0
Recall: 0.375
F1-score: 0.5454545454545454
------------
Performace of  I-PER
Precision: 0.5
Recall: 0.2857142857142857
F1-score: 0.36363636363636365
------------
Performace of  IPER
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  I-LOC
Precision: 0
Recall: 0.0
F1-score: 0
------------
Performace of  

In [ ]:
print("Macro Precision:", precision)
print("Macro Recall:", recall)
print("Macro F1-score:", f1)

Macro Precision: 0.5262703799468506
Macro Recall: 0.45773404599935213
Macro F1-score: 0.47501443001442994


# **Question 5**

# Performance Comparison of Indic-NER and Indic-BERT Models for Named Entity Recognition

## Introduction
In this report, we compare the performance of two state-of-the-art models, Indic-NER and Indic-BERT, for the task of Named Entity Recognition (NER). NER is the task of identifying and classifying named entities such as persons, organizations, locations, and miscellaneous entities in text data.

## Hyperparameters Tuned
We tuned several hyperparameters during training to optimize the performance of both models:
- **Learning Rate:** Set to 2e-5 for gradual updates to model parameters.
- **Batch Size:** Fixed at 8 to balance training speed and memory consumption.
- **Number of Training Epochs:** Set to 3 to allow the model to learn from the data without overfitting.
- **Weight Decay:** Set to 0.01 for moderate regularization to prevent overfitting.
- **Evaluation Strategy:** Evaluated after each epoch to monitor model performance throughout training.

## Results
### Performance of Indic-NER Model
- **Macro Precision:** 0.6953
- **Macro Recall:**  0.48639
- **Macro F1-score:** 0.5372
#### Individual Named Entity Classes:
- **B-LOC:**
  - Precision: 0.667, Recall: 0.25, F1-score: 0.364
- **B-ORG:**
  - Precision: 0.556, Recall: 0.833, F1-score: 0.667
- **O:**
  - Precision: 0.883, Recall: 0.994, F1-score: 0.936
- **B-PER:**
  - Precision: 0.933, Recall: 0.933, F1-score: 0.933
- **Other classes (B-MISC, I-MISC, I-ORG, I-PER, I-LOC):**
  - Precision, Recall, and F1-score: 0 (due to no correct predictions)

### Performance of Indic-BERT Model
- **Macro Precision:** 0.52284
- **Macro Recall:** 0.50583
- **Macro F1-score:** 0.4999


#### Individual Named Entity Classes:
- **B-ORG:**
  - Precision: 0.800, Recall: 0.667, F1-score: 0.727
- **I-ORG:**
  - Precision: 0.750, Recall: 1.000, F1-score: 0.857
- **O:**
  - Precision: 0.936, Recall: 0.974, F1-score: 0.954
- **Other classes (B-MISC, I-MISC, B-PER, B-LOC, I-PER, I-LOC):**
  - Precision, Recall, and F1-score: 0 (due to no correct predictions)

## Comparison and Discussion
- The Indic-NER model achieved a slightly higher macro F1-score compared to the Indic-BERT model (0.5372 vs. 0.4999), indicating better overall performance.
- However, both models exhibited varying performance across individual named entity classes.
- The Indic-BERT model outperformed the Indic-NER model in certain classes (e.g., B-ORG, I-ORG), while the Indic-NER model performed better in others (e.g., B-PER, I-PER).
- Further analysis is required to understand the factors influencing model performance and to identify areas for improvement.

## Conclusion
In conclusion, the Indic-NER model showed better overall performance for the task of Named Entity Recognition compared to the Indic-BERT model. However, both models have their strengths and weaknesses, and further research is needed to enhance their performance and address specific challenges in NER for Indian languages.
